# Hyperlinks' clickability preferences

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.parse

In [2]:
import os, sys

# HTLM files URLs in a list 
wp_path = "/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/" # to adapt
directories = os.listdir(wp_path)
html_list = []
for dir in directories:
    local_path = wp_path + dir
    local_dir = os.listdir(local_path)
    for link in local_dir:
        if link[-3:]=='htm':
            html_list.append(dir+'/'+link)
html_list[:10]

['r/Royal_Marines.htm',
 'r/Recycling.htm',
 'r/Retreat_of_glaciers_since_1850.htm',
 'r/Remember_Girls_Like_Rajpura.htm',
 'r/Rutherfordium.htm',
 'r/Rule_of_St_Benedict.htm',
 'r/Red_dwarf.htm',
 'r/Romeo_and_Juliet.htm',
 'r/Rudyard_Kipling.htm',
 'r/Race.htm']

## Objective

The aim of this specific section of our game strategy analysis is to test whether Wikispeedia players have clikability preferences on the basis of the hyperlinks' positioning throughout the article text. 
In particular:
1. we wil find the ordinal number of the paragraph each hyperlink in an article shows up in and divide it by the total number of paragraphs of the article, to determine its positioning; 
2. we will determine if each hyperlink in an article shows up in an image captions.

We will then investigate if there is a correlation between the clickability of the hyperlinks and their location, using the proper statistical tests. 

### Hyperlinks positioning in articles' text

It is essential to extract from the data the frequency of clicks for each hyperlink.

In [3]:
# read files 
folder='/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wikispeedia_paths-and-graph/'

paths_finished='paths_finished.tsv'
paths_unfinished='paths_unfinished.tsv'
articles='articles.tsv'

paths_finished=pd.read_csv(folder+paths_finished, sep='\t', skiprows=15, header=None, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'])
paths_unfinished=pd.read_csv(folder+paths_unfinished, sep='\t', skiprows=16, header=None, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'target', 'type'])
articles=pd.read_csv(folder+articles, sep='\t', skiprows=12, header=None, names=['article'])

The dataframe `paths_finished` contains the paths chosen by the players who could reach the target article.

In [4]:
paths_finished.head()

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0


The dataframe `paths_unfinished` contains the paths chosen by the players who could not reach the target article.

In [5]:
paths_unfinished.head()

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout


The dataframe `articles` lists all the clickable articles/links.

In [6]:
# encode and filter link names
def filter_link_list(links_list):
    new_links_list=[]
    for link in links_list:
        link=urllib.parse.unquote(link)
        link=link.lower()
        link=link.replace('_', ' ')
        link=link.strip()
        new_links_list.append(link)
    return new_links_list

articles.article=filter_link_list(articles.article)
articles.head(10)

,article
0,áedán mac gabráin
1,åland
2,édouard manet
3,éire
4,óengus i of the picts
5,€2 commemorative coins
6,10th century
7,11th century
8,12th century
9,13th century


In [7]:
print('The total number of articles comprising the Wikispeedia library is {0}.'.format(len(articles)))

The total number of articles comprising the Wikispeedia library is 4604.


The dataframe `clicked_links_filtered` contains the clicked links troughout the paths.

In [8]:
clicked_links=[]
for x in paths_finished['path']: 
    path=(x.split(';')) # get list of links along the path string 
    clicked_links.append(path)

clicked_links_filtered=[] # consistent with other dfs
for path in clicked_links:
    path_filtered=filter_link_list(path)
    if '<' in path: 
        path.remove('<') # remove backclicks
    clicked_links_filtered.append(path_filtered)

clicked_links_filtered[:5]

[['14th century',
  '15th century',
  '16th century',
  'pacific ocean',
  'atlantic ocean',
  'accra',
  'africa',
  'atlantic slave trade',
  'african slave trade'],
 ['14th century',
  'europe',
  'africa',
  'atlantic slave trade',
  'african slave trade'],
 ['14th century',
  'niger',
  'nigeria',
  'british empire',
  'slavery',
  'africa',
  'atlantic slave trade',
  'african slave trade'],
 ['14th century', 'renaissance', 'ancient greece', 'greece'],
 ['14th century',
  'italy',
  'roman catholic church',
  'hiv',
  'ronald reagan',
  'president of the united states',
  'john f. kennedy']]

The dataframe `links_freq_df` contains the links' frequency of clicks.

In [ ]:
links_name=[]
links_freq=[]

i=0
for article in articles['article']: # count the number of times each article shows up in paths
    count=0
    links_name.append(article)
    for i in range(len(clicked_links_filtered)):
        if article in clicked_links_filtered[i]:
            count+=1 
    links_freq.append(count)

links_name=pd.Series(links_name)
links_freq=pd.Series(links_freq)
links_freq_df=pd.concat([links_name, links_freq], axis=1, names=['link', '#_clicks'])
links_freq_df.columns=['link', '#_clicks']
links_freq_df.loc[links_freq_df['#_clicks']!=0].head()

In [ ]:
clicks_tot=links_freq_df['#_clicks'].sum()
print('The total number of clicked hyperlinks is {0}.'.format(clicks_tot))

Next step of the descriptive analysis process is the html files parsing aimed at determining the location of hyperlinks. 

In [ ]:
# parse html files 

from bs4 import BeautifulSoup

soups=[]
failed_html_list = []
for html in html_list[:]:
    f=open(wp_path+html, 'r')
    try:
        soup = BeautifulSoup(f, 'lxml')
        soups.append(soup)
    except Exception:
        failed_html_list.append(html)

In [ ]:
print('{0} files were not parsable. However, they are not reachable nor it is possible to click \
any hyperlink in those html pages.'.format(len(failed_html_list)))

`link_positioning_dfs` is a list of dataframes (`link_positioning_df`) containing hyperlinks showing up in one html file, the ordinal number of the paragraph they belong to and their relative positioning throughout the article text. 

In [ ]:
# HTML tag for hyperlinks is <a href= >
# the function finds all <a href= > tags throughout the article html file

def find_links(soup): 
    paragraphs = soup.find_all('p') 
    links=[]
    i=0
    for p in paragraphs:
        i+=1
        all_links=p.find_all('a')
        for link in all_links:
            if 'href' in link.attrs:
                link_title=link.text
                link_paragraph=i
                links.append([link_title, link_paragraph])
                
    link_positioning_df=pd.DataFrame(links, columns=['link', '#_paragraph']).drop_duplicates()
    link_positioning_df['positioning']=round(link_positioning_df['#_paragraph']/len(paragraphs), 2)
    link_positioning_df['link']=filter_link_list(link_positioning_df['link'])
    return link_positioning_df

In [ ]:
# apply the function to all html files

link_positioning_dfs=[]
for soup in soups:
    link_positioning_dfs.append(find_links(soup))

link_positioning_dfs[:5]

In [ ]:
print('{0} html files were parsed.'.format(len(link_positioning_dfs)))

In [ ]:
# example with 2 files

URL_1 = wp_path+'a/A_cappella.htm'
URL_2 = wp_path+'m/Music.htm'

f1=open(URL_1, 'r')
soup1 = BeautifulSoup(f1, 'lxml')

f2=open(URL_2, 'r')
soup2 = BeautifulSoup(f2, 'lxml')

link_positioning_df_1=find_links(soup1).drop(labels='#_paragraph', axis=1)
link_positioning_df_2=find_links(soup2).drop(labels='#_paragraph', axis=1)

# merging the 2 dataframes 

merged_dfs=link_positioning_df_1.merge(link_positioning_df_2, how='outer', left_on='link', right_on='link', )
merged_dfs

The hyperlink _rap_ shows up in both articles; the hyperlink _jazz_, _renaissance_ and _time_ show up multiple times in one article. 

The two occurrences highlighted are likely to be the case for every hyperlink, when all dataframes will be merged. It is therefore essential to compute their most likely positioning in the articles' texts, so to extract any meaningful information from the data.  

In [ ]:
# merge all the link_positioning_df in link_positioning_dfs list on hyperlink in order to apply a function 
# that determines the most frequent positioning of the hyperlinks

for df in link_positioning_dfs:
    try:
        df.drop(labels='#_paragraph', axis=1, inplace=True)  # remove not needed column
    except:
        pass    

    
from functools import reduce # perform multiple merge
links_pos_dfs_merged = reduce(lambda  left,right: pd.merge(left,right,on=['link'],how='outer'), link_positioning_dfs[:100])
links_pos_dfs_merged.head()

In [ ]:
# function that determines the most frequent positioning of each hyperlink in the set of all links

def find_positioning(array):
    bin_labels=['top', 'center-top','center','center-bottom', 'bottom'] # positioning is classified as top, centre-top, center, center-bottom, or bottom of the article
    bins=[0, 0.20, 0.40, 0.60, 0.80, 1] 
    cut_positioning=pd.cut(array, bins, labels=bin_labels, right=True, include_lowest=True)
    positioning_frequency=pd.value_counts(cut_positioning) # frequency of the bins
    most=positioning_frequency.apply('max') # bin with highest frequency
    if len(positioning_frequency[positioning_frequency==most])>1:
        return np.random.choice(positioning_frequency[positioning_frequency==most].index.values.astype(str)) #account for same frequency
    else:
        return positioning_frequency[positioning_frequency==most].index.values.astype(str).item()

The dataframe `most_freq_positioning_df` contains the hyperlinks' most frequent positioning.

In [ ]:
# example with 2 files

most_freq_positioning_2=merged_dfs.groupby(by=merged_dfs['link']).apply(lambda x: find_positioning(np.array(x.values).flatten()))
most_freq_positioning_df_2=pd.DataFrame(most_freq_positioning_2, columns=['most_freq_positioning'])
most_freq_positioning_df_2=most_freq_positioning_df_2.reset_index()
most_freq_positioning_df_2

In [ ]:
# all html files

most_freq_positioning=links_pos_dfs_merged.set_index("link").groupby("link").apply(lambda x: find_positioning(x.values.flatten()))
most_freq_positioning_df=pd.DataFrame(most_freq_positioning, columns=['most_freq_positioning'])
most_freq_positioning_df

The dataframe `links_freq_pos` contains both the hyperlinks' most frequent positioning and their frequency of clicks.

In [ ]:
links_freq_pos=links_freq_df.merge(most_freq_positioning_df, how='outer', left_on='link', right_on='link')
links_freq_pos

The dataframe `links_freq_pos_grouped_df` groups the hyperlinks click frequency by their positioning.

In [ ]:
links_freq_pos_grouped=links_freq_pos['#_clicks'].groupby(links_freq_pos['most_freq_positioning']).sum()
links_freq_pos_grouped_df=pd.DataFrame(links_freq_pos_grouped)
links_freq_pos_grouped_df

In [ ]:
# normalization (/clicks_tot if possible to merge all files)

links_freq_pos_grouped_df['#_clicks_norm']=links_freq_pos_grouped_df['#_clicks']/(links_freq_pos_grouped_df.sum().values)
links_freq_pos_grouped_df

In order to compare the hyperlinks click frequencies with regard to their positioning (i.e. _top_, _center-top_, _center_, _center-bottom_, _bottom_) their absolute values need to be normalized with the bins' frequencies (i.e. the number of times the hypelinks actually show up in the categorical positioning).

The dataframe `positioning_freq_df_grouped` contains the frequency of hyperlinks showing up in the Wikispeedia set of articles grouped by their positioning throughout the text. 

In [ ]:
# function that counts the frequency of a link showing up at the top, center, bottom of the article

def find_positioning_dis(array):
    bin_labels=['top', 'center-top','center', 'center-bottom', 'bottom']
    bins=[0, 0.2, 0.4, 0.6, 0.8, 1]
    cut_positioning=pd.cut(array, bins, labels=bin_labels, right=True, include_lowest=True)
    positioning_frequency=pd.value_counts(cut_positioning)
    return positioning_frequency

In [ ]:
# apply the function to the dataframe links positioning grouped by link  

positioning_freq=links_pos_dfs_merged.set_index("link").groupby("link").apply(lambda x: find_positioning_dis(x.values.flatten()))
positioning_freq_df=pd.DataFrame(positioning_freq, columns=['positioning_freq'])
positioning_freq_df.reset_index(level=1, inplace=True)
positioning_freq_df.rename(columns={'level_1':'positioning'}, inplace=True)
positioning_freq_df.head(20)

In [ ]:
positioning_freq_df_grouped=positioning_freq_df.groupby('positioning').sum()
positioning_freq_df_grouped.columns=['nr']
links_tot=np.sum(positioning_freq_df['positioning_freq'])
positioning_freq_df_grouped['nr_norm']=positioning_freq_df_grouped['nr']/links_tot
positioning_freq_df_grouped

In [ ]:
# visualization of the distribution of the clicked hyperlinks' positioning

# dataframe for visualization
vis=links_freq_pos_grouped_df.merge(positioning_freq_df_grouped, left_index=True, right_index=True)

fig, ax = plt.subplots(1,2, figsize=(11,4))
vis.plot(y=['#_clicks_norm', 'nr_norm'], kind='bar', ax=ax[0], label=['#_clicks', '#_hyperlinks'], color=['orange', 'green'])
ax[0].legend()
ax[0].set_title('Distribution of the hyperlinks positioning')
ax[0].set_xlabel('Positioning')
ax[0].set_ylabel('Frequency normalized')

vis['clicked']=vis['#_clicks_norm']/vis['nr_norm'] # not sure
vis.plot(y='clicked', kind='bar', ax=ax[1], color='pink', label='')
ax[1].legend()
ax[1].set_title('Distribution of the hyperlinks clickability')
ax[1].set_xlabel('Positioning')
ax[1].set_ylabel('#_clicks / #_hyperlinks')

The bar plot on the left illustrates the distribution of the clicked hyperlinks and the total number of links in the articles, on the basis of their positioning throughout the articles text, both normalized. 

The bar plot on the right illustrates the hyperlinks clickability on the basis of their positioning. More precisely: 
1. a value of 1 (_\#_clicks_/_\#_hyperlinks_ = 1) corresponds to a hyperlinks clickability on average equally proportional to the normalized frequency of the specific categorical positioning (i.e. the number of clicks proportional to the number times a hyperlink shows up in a section of the article);
2. a value of greater than 1, points a more than proportional clickability;
3. a value of smaller than 1, points a less than proportional clickability.

### Hyperlinks positioning in images' caption

The following part of the descriptive analysis focuses on the location of hyperlinks in images' captions.

Images in html files can be either found under \<div> or \<table> tag, while the hyperlinks are found in their captions under \<div class='thumbcaption'> tag.

The dataframe `links_in_tables` contains the links showing up in tables' captions. 

In [ ]:
# parse html file and search hyperlinks in the captions of tables containing images 

def find_link_in_table(soup):
    
    tables = soup.find_all('table')

    links=[]
    for table in tables:
        if table.find('img'):
            all_links=table.find_all('a')
            for link in all_links:
                if 'class' not in link.attrs:
                    link_title=link.text
                    links.append(link_title)

    if links:
        links_in_table_df=pd.DataFrame(links).drop_duplicates()
        links_in_table_df.columns=['link']
        links_in_table_df=links_in_table_df.apply(filter_link_list) 
    else: 
        return None
    return links_in_table_df

In [ ]:
links_in_tables=[]
for soup in soups:
    links_in_tables.append(find_link_in_table(soup))

links_in_tables=pd.concat(links_in_tables, axis=0)
links_in_tables.head()

In [ ]:
links_in_tables_tot=len(links_in_tables)
print('The total number of links in tables captions is {0}.'.format(links_in_tables_tot))

The dataframe `links_in_images` contains the links showing up in images' captions.

In [ ]:
# parse html file and search hyperlinks in the captions of div containing images 

def find_links_in_img(soup):
    div = soup.find_all('div')

    links=[]

    for d in div:
        if (d.find('a', class_="internal")):
            if (d.find('img', class_='thumbimage')):
                caption=d.find('div', class_="thumbcaption")
                if (caption.find('a')):
                    all_links=caption.find_all('a')
                    for link in all_links:
                        if 'class' not in link.attrs:
                            link_title=link.text
                            links.append(link_title)
    if links:
        links_in_image_df=pd.DataFrame(links).drop_duplicates()
        links_in_image_df.columns=['link']
        links_in_image_df=links_in_image_df.apply(filter_link_list) 
    else: 
        return None
        links_in_image_df=pd.DataFrame()
    return links_in_image_df

In [ ]:
links_in_images=[]
for soup in soups:
    links_in_images.append(find_links_in_img(soup))

links_in_images=pd.concat(links_in_images, axis=0)
links_in_images.head()

In [ ]:
links_in_images_tot=len(links_in_images)
print('The total number of links in images captions is {0}.'.format(links_in_images_tot))

Next, the supposed clickability preference for a hyperlink showing up in an image will be investigated, by grouping the click frequency by hyperlinks' location.

In [ ]:
# drop indexes to ease the flagging
links_in_images=links_in_images.reset_index(drop=True).drop_duplicates()
links_in_tables=links_in_tables.reset_index(drop=True).drop_duplicates()

The dataframe `links_freq_img` contains hyperlinks click frequency and a flag specifying if they can be found in images' captions.

In [ ]:
# link in image

flag_img=[]
links_freq_img=links_freq_df.copy()

for link in links_freq_img['link']:
    if link in list(links_in_images['link']):
        flag_img.append(1)
    else:
        flag_img.append(0)
    
links_freq_img['in_image']=flag_img
links_freq_img[links_freq_img['in_image']==1][:10]

The dataframe `links_freq_tab` contains hyperlinks click frequency and a flag specifying if they can be found in tables' captions.

In [ ]:
# link in table

flag_tab=[]
links_freq_tab=links_freq_df.copy()

for link in links_freq_tab['link']:
    if link in list(links_in_tables['link']):
        flag_tab.append(1)
    else:
        flag_tab.append(0)
    
links_freq_tab['in_tab']=flag_tab
links_freq_tab[links_freq_tab['in_tab']==1][:10]

The dataframe `links_freq_img_grouped_df` contains the frequency of clicks for hyperlinks showing up in images' captions and not, and their normalized values.

In [ ]:
links_freq_img_grouped_df=pd.DataFrame(links_freq_img['#_clicks'].groupby(links_freq_img['in_image']).sum())
links_freq_img_grouped_df.loc[0,'#_clicks_norm_img']=links_freq_img_grouped_df.loc[0,'#_clicks']/clicks_tot
links_freq_img_grouped_df.loc[1,'#_clicks_norm_img']=links_freq_img_grouped_df.loc[1,'#_clicks']/clicks_tot

links_freq_img_grouped_df.loc[0,'#_clicks/#_links_img']=links_freq_img_grouped_df.loc[0,'#_clicks_norm_img']/(links_tot-links_in_images_tot)
links_freq_img_grouped_df.loc[1,'#_clicks/#_links_img']=links_freq_img_grouped_df.loc[1,'#_clicks_norm_img']/links_in_images_tot

links_freq_img_grouped_df

The dataframe `links_freq_tab_grouped_df` contains the frequency of clicks for hyperlinks showing up in tables' captions and not, and their normalized values.

In [ ]:
links_freq_tab_grouped_df=pd.DataFrame(links_freq_tab['#_clicks'].groupby(links_freq_tab['in_tab']).sum())
links_freq_tab_grouped_df.loc[0,'#_clicks_norm_tab']=links_freq_tab_grouped_df.loc[0,'#_clicks']/clicks_tot
links_freq_tab_grouped_df.loc[1,'#_clicks_norm_tab']=links_freq_tab_grouped_df.loc[1,'#_clicks']/clicks_tot

links_freq_tab_grouped_df.loc[0,'#_clicks/#_links_tab']=links_freq_tab_grouped_df.loc[0,'#_clicks_norm_tab']/(links_tot-links_in_tables_tot)
links_freq_tab_grouped_df.loc[1,'#_clicks/#_links_tab']=links_freq_tab_grouped_df.loc[1,'#_clicks_norm_tab']/links_in_tables_tot

links_freq_tab_grouped_df

In [ ]:
vis_img=links_freq_img_grouped_df.merge(links_freq_tab_grouped_df, left_index=True, right_index=True)
vis_img

In [ ]:
# dataframe for visualization
vis_img=links_freq_img_grouped_df.merge(links_freq_tab_grouped_df, left_index=True, right_index=True)

fig, ax = plt.subplots(1,2, figsize=(10,4), sharey=True)

links_freq_img_grouped_df.plot(y=['#_clicks_norm_img'], kind='bar', ax=ax[0], label=['image'])
links_freq_tab_grouped_df.plot(y=['#_clicks_norm_tab'], kind='bar', ax=ax[1], label=['table'])

ax[0].legend()
ax[1].legend()
ax[0].set_title('Distribution of clicked hyperlinks')
ax[1].set_title('Distribution of clicked hyperlinks')
ax[0].set_xlabel('Link in image caption')
ax[1].set_xlabel('Link in table caption')
ax[0].set_ylabel('Frequency normalized')

fig, ax = plt.subplots(figsize=(6,5))
vis_img.plot(y=['#_clicks/#_links_img', '#_clicks/#_links_tab'], kind='bar', ax=ax, label=['image', 'table'], color=['orange', 'green'])
ax.legend()
ax.set_xlabel('Link in image or table caption')

In [ ]:
# need comment

Assumptions: 
1. when a hyperlink shows up in both the article text and in an image caption, it is not possible to extract from the data in our possess what the player's click choice perfomed is. Therefore we assumed that hyperlinks showing up in images' caption don't appear in the text.
2. for the hyperlinks which show up multiple times in the article ...